# 🚀 Proyecto Churn — Telecom X (LATAM)

### Notebook: Análisis, limpieza y preparación de datos para modelado

Este notebook toma como entrada el archivo `TelecomX_Data.json`, realiza una exploración, limpieza y preparación de los datos (feature engineering y exportación). El resultado es un dataset limpio listo para que el equipo de Ciencia de Datos pueda construir modelos predictivos de churn.

**Instrucciones:**
- Ejecuta todas las celdas en orden.
- El notebook genera un archivo `TelecomX_Data_clean.csv` con los datos limpios.

Desarrollado como solución descargable para el cliente.

In [ ]:
# 1. Librerías necesarias
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set(style='whitegrid', palette='pastel', font_scale=1.05)
plt.rcParams['figure.figsize'] = (10,6)
print('Librerías cargadas')

## 2. Cargar los datos

Se asume que el archivo `TelecomX_Data.json` está en el mismo directorio que este notebook o en `/mnt/data/`.

In [ ]:
# Intentamos localizar el archivo en rutas comunes
candidates = [
    'TelecomX_Data.json',
    '/mnt/data/TelecomX_Data.json',
    './data/TelecomX_Data.json'
]
data_path = None
for p in candidates:
    if Path(p).exists():
        data_path = Path(p)
        break

if data_path is None:
    raise FileNotFoundError('No se encontró TelecomX_Data.json. Coloca el archivo en el mismo directorio que el notebook o en /mnt/data/')
else:
    print(f'Archivo encontrado en: {data_path}')

# Cargar JSON a DataFrame (maneja formatos de lista de registros o línea por línea)
try:
    df = pd.read_json(data_path)
except ValueError:
    # fallback a read_json orient='records' o lines=True
    try:
        df = pd.read_json(data_path, lines=True)
    except Exception as e:
        raise

print('Datos cargados en DataFrame con forma:', df.shape)
display(df.head())

## 3. Inspección rápida
Revisamos tipos, valores nulos y descripción estadística para entender la estructura.

In [ ]:
print('Tipos de datos:')
display(df.dtypes)

print('\nResumen de valores nulos por columna:')
display(df.isna().sum())

print('\nDescripción estadística (numérica):')
display(df.describe(include=[np.number]))

print('\nDescripción (objetos/categorías):')
display(df.describe(include=['object', 'category']))

## 4. Objetivo: identificar la variable de churn

Buscamos columnas que indiquen abandono del cliente (por ejemplo `churn`, `is_churn`, `cancelled`, `status`, etc.).

In [ ]:
# Buscar columnas que puedan indicar churn
possible_churn_cols = [c for c in df.columns if 'churn' in c.lower() or 'cancel' in c.lower() or 'status' in c.lower() or 'aband' in c.lower()]
print('Columnas candidatas a churn:', possible_churn_cols)
display(df[possible_churn_cols].head()) if possible_churn_cols else print('No se detectaron columnas con nombres obvios para churn.')

Si no existe una columna obvia, habrá que crear una variable objetivo basada en reglas (por ejemplo, `contract_end` o `days_without_service`). En este notebook seguiremos ambos enfoques: si existe `churn` la usaremos; si no, intentaremos inferirla.

In [ ]:
# Definimos la variable objetivo 'churn' si no existe
lower_cols = [c.lower() for c in df.columns]
if 'churn' in lower_cols:
    churn_col = [c for c in df.columns if c.lower()=='churn'][0]
    df['churn'] = df[churn_col].astype(int)
    print('Se usó la columna existente:', churn_col)
elif possible_churn_cols:
    # si hay columnas candidatas, escogemos la primera (solo heurística)
    df['churn'] = df[possible_churn_cols[0]].apply(lambda x: 1 if str(x).lower() in ['yes','y','true','1','si','s','cancelled'] else 0)
    print('Se creó churn a partir de:', possible_churn_cols[0])
else:
    print('No se detectó columna de churn. Se intentará inferir a partir de columnas de estado o de fecha (si existe).')
    if 'status' in df.columns:
        df['churn'] = df['status'].apply(lambda x: 1 if str(x).lower() in ['inactive','cancelled','suspended','terminated'] else 0)
    else:
        df['churn'] = 0
        print('Se estableció churn=0 por defecto: revisa manualmente')

display(df[['churn']].head())


## 5. Limpieza básica
- Normalizar nombres de columnas
- Tratar valores nulos
- Convertir tipos
- Detectar y tratar outliers sencillos

In [ ]:
# Normalizar columnas: minusculas y reemplazar espacios
df.columns = [c.strip().lower().replace(' ', '_').replace('/', '_') for c in df.columns]
print('Columnas normalizadas')
display(df.columns)

# Mostrar nulos porcentuales
nulos = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
display(nulos.head(20))


In [ ]:
# Estrategia de imputación (heurística) - se aporta como plantilla
df_clean = df.copy()
num_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()

print('Columnas numéricas:', num_cols)
print('Columnas categóricas:', cat_cols)

for c in num_cols:
    if df_clean[c].isna().sum() > 0:
        med = df_clean[c].median()
        df_clean[c].fillna(med, inplace=True)

for c in cat_cols:
    if df_clean[c].isna().sum() > 0:
        df_clean[c].fillna('missing', inplace=True)

print('Imputación realizada. Nulos restantes por columna:')
display(df_clean.isna().sum()[df_clean.isna().sum()>0])


## 6. Análisis exploratorio básico (EDA)
Calculamos tasas de churn, visualizamos distribuciones y relaciones entre variables.

In [ ]:
churn_rate = df_clean['churn'].mean()
print(f'Tasa de churn (porcentaje): {churn_rate*100:.2f}%')

sns.countplot(x='churn', data=df_clean)
plt.title('Distribución de churn (0 = activo, 1 = churn)')
plt.show()

sample_cats = [c for c in cat_cols if c not in ['id', 'customer_id']][:5]
for c in sample_cats:
    plt.figure()
    ct = pd.crosstab(df_clean[c], df_clean['churn'], normalize='index')*100
    ct.plot(kind='bar', stacked=True)
    plt.title(f'Churn por {c}')
    plt.ylabel('Porcentaje por fila')
    plt.legend(title='churn')
    plt.show()


In [ ]:
if len(num_cols) > 0:
    corr = df_clean[num_cols + ['churn']].corr()
    plt.figure(figsize=(12,8))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0)
    plt.title('Mapa de correlación (numéricas + churn)')
    plt.show()
else:
    print('No hay columnas numéricas para calcular correlación')


## 7. Feature engineering rápido
- Convertir fechas a datetime
- Crear variables simples (tenure en meses, totales, promedios)

Se muestra una plantilla: adapta según las columnas reales del dataset.

In [ ]:
if 'signup_date' in df_clean.columns and 'last_active_date' in df_clean.columns:
    df_clean['signup_date'] = pd.to_datetime(df_clean['signup_date'], errors='coerce')
    df_clean['last_active_date'] = pd.to_datetime(df_clean['last_active_date'], errors='coerce')
    df_clean['tenure_days'] = (df_clean['last_active_date'] - df_clean['signup_date']).dt.days
    df_clean['tenure_months'] = (df_clean['tenure_days'] / 30).round(1)
    print('Se creó tenure_months')
else:
    print('No se detectaron columnas de fecha para crear tenure. Revisar manualmente.')


## 8. Preparar dataset final para modelado
- Codificar variables categóricas (label encoding / one-hot)
- Seleccionar features relevantes
- Exportar CSV limpio

In [ ]:
df_model = df_clean.copy()
cat_small = [c for c in cat_cols if df_model[c].nunique() <= 10 and c != 'churn']
print('Categorías a one-hot:', cat_small)
df_model = pd.get_dummies(df_model, columns=cat_small, drop_first=True)
df_model['churn'] = df_model['churn'].astype(int)
out_path = Path('TelecomX_Data_clean.csv')
df_model.to_csv(out_path, index=False)
print(f'Dataset limpio guardado en: {out_path} con shape {df_model.shape}')
display(df_model.head())


## 9. Recomendaciones y siguientes pasos

1. Revisar manualmente la definición de `churn` si fue inferida por heurística.
2. Realizar análisis de balanceo (SMOTE / undersampling) si la clase churn está desbalanceada.
3. Avanzar con pipeline de modelado: separación train/test, validación cruzada y pruebas con modelos (Logistic Regression, Random Forest, XGBoost).
4. Documentar transformaciones aplicadas (importantísimo para producción y reproducibilidad).

Esto entrega al equipo de ciencia de datos un CSV listo para entrenar modelos predictivos.
